# See how well it did on real data
The spectra are real. We have results from `ssv` files from SpectrumMill. We can load this and see if the sequences we identified matches the sequences SpectrumMill identified, plus the hybrid ones

## 1. Load the `ssv` file

In [1]:
import pandas as pd
import json

specmilresultsfile = '/Users/zacharymcgrath/Downloads/NOD2_E3_results.ssv'
specmilresults = pd.read_csv(specmilresultsfile, sep=';')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/zacharymcgrath/Downloads/NOD2_E3_results.ssv'

In [2]:
specmilresults.loc[specmilresults['filename'] == 'NOD2_E3.7065.7065.2']

number             filename  parent_charge  score  \
6       7  NOD2_E3.7065.7065.2              2  12.55   

   deltaForwardReverseScore  deltaRank1Rank2Score  \
6                      6.28                  7.14   

   percent_scored_peak_intensity  totalIntensity previous_aa     sequence  \
6                           81.3        282000.0         (G)  DLPVNSPMTKG   

  next_aa  retentionTimeMin  chromatographicPeakWidthSec  parent_m_over_z  \
6     (D)             26.08                         15.0         579.7953   

  species                                         entry_name  
6   MOUSE  HYBRID: mouse ins2C EVEDPQVAQLELGGGPGAGD-LPVNS...

### 1.a Get the hybrids out of the dataframe

In [3]:
specmilhybs = specmilresults.loc[specmilresults['entry_name'].str.contains('HYBRID')]
filenameindexedhybs = {row['filename']: row['sequence'] for _, row in specmilhybs.iterrows()}


### 1.b Get the non hybrids out of the dataframe

In [4]:
specmilnonhybs = specmilresults.loc[~specmilresults['entry_name'].str.contains('HYBRID')]
filenameindexednonhybs = {row['filename']: row['sequence'] for _, row in specmilnonhybs.iterrows()}

## 2. Load the results from hypedsearch

In [5]:
hyped_res_dir = '/Users/zacharymcgrath/Desktop/Experiment output/filtered_NOD2_E3_SEP_9/'

nonhybridresultsfile = hyped_res_dir + 'summary.tsv'
hybridresultsfile = hyped_res_dir + 'hybrid_summary.tsv'
resultsjsonfile = hyped_res_dir + 'summary.json'

nonhybridresults = pd.read_csv(nonhybridresultsfile, sep='\t')
hybridresults = pd.read_csv(hybridresultsfile, sep='\t')


In [6]:
nonhybridresults.head()

Unnamed: 0                                           proteins    sequence  \
0           0  ['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...   DPQVEQLEL   
1           1  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...  DLQTLALEVA   
2           2                                                 []   TTPGTLALE   
3           3  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...     DLQTLAL   
4           4                                                 []  DLQTLALEVE   

   b_score  y_score  total_score  precursor_distance  entry name  \
0        7        2            9            0.000673           0   
1        7        2            9            0.002875           1   
2        6        3            9            0.000877           2   
3        4        2            6            0.000438           3   
4        7        4           11            0.004477          11   

                          id  
0  NOD2_E3.13446.13477.2.pkl  
1  NOD2_E3.18005.18246.2.pkl  
2  NOD2_E3.13729.13828.2.pkl  
3  NOD2_E3.15226.15503.2.pkl  
4  NOD2_E3.16920.17393.2.pkl

In [7]:
hybridresults.head()

Unnamed: 0                                      left_proteins  \
0           0  ['sp|Q91ZI0|CELR3_MOUSE Cadherin EGF LAG seven...   
1           1  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...   
2           2  ['sp|Q00623|APOA1_MOUSE Apolipoprotein A-I OS=...   
3           3  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...   
4           4  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...   

                                      right_proteins  \
0  ['sp|P62500|T22D1_MOUSE TSC22 domain family pr...   
1  ['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...   
2  ['sp|P26339|CMGA_MOUSE Chromogranin-A OS=Mus m...   
3  ['sp|Q6GQX6|ANKS6_MOUSE Ankyrin repeat and SAM...   
4  ['sp|NOD0001|IAPP_MOUSE NOD Islet amyloid poly...   

                           sequence                    hybrid_sequence  \
0                        TLAITASPLI                        TLAITA-SPLI   
1  DPQVAQLELGGGPGAGDLQTAQLELGGGPGAG  DPQVAQLELGGGPGAGDLQT-AQLELGGGPGAG   
2                       LDPVNSPMTKG                       LD-PVNSPMTKG   
3                       DLQTLALWSRM                       DLQTLAL-WSRM   
4                       DLQTLALNAAR                       DLQTLAL-NAAR   

   b_score  y_score  total_score  precursor_distance  entry name  \
0        5        3            8            0.000194           4   
1       16        8           24            0.505994           5   
2        2        5            7            0.000452           6   
3        5        8           13            0.000192           7   
4        4        9           13            0.000812           8   

                          id  
0  NOD2_E3.21510.21510.2.pkl  
1  NOD2_E3.12771.12902.3.pkl  
2    NOD2_E3.7065.7065.2.pkl  
3  NOD2_E3.16373.16401.2.pkl  
4  NOD2_E3.10614.10681.2.pkl

## 3. Check to see the overlap in results

In [8]:
# pairs are (hypedsearch, specmil, id)
untracked = []
nonhybpairs = []
for _, row in nonhybridresults.iterrows():
    fname = row['id'].replace('.pkl', '')
    if fname in filenameindexednonhybs:
        nonhybpairs.append((row['sequence'], filenameindexednonhybs[fname], fname, 'non-hybrid'))
    elif fname in filenameindexedhybs:
        nonhybpairs.append((row['sequence'], filenameindexedhybs[fname], fname, 'hybrid'))
    else:
        untracked.append((row['sequence'], fname, '---'))

hybpairs = []
for _, row in hybridresults.iterrows():
    fname = row['id'].replace('.pkl', '')
    if fname in filenameindexednonhybs:
        hybpairs.append((row['sequence'], filenameindexednonhybs[fname], fname, 'non-hybrid'))
    elif fname in filenameindexedhybs:
        hybpairs.append((row['sequence'], filenameindexedhybs[fname], fname, 'hybrid'))
    else:
        untracked.append((row['sequence'], fname, '---'))

untracked

[]

In [9]:
gc = 0
missed_nhpairs = []
for pair in nonhybpairs:
    # replace LI with B 
    hyped_LI_less = pair[0].replace('L', 'B').replace('I', 'B')
    specmil_LI_less = pair[1].replace('L', 'B').replace('I', 'B')
    if hyped_LI_less != specmil_LI_less:
        missed_nhpairs.append(pair)
        print(pair)
    else:
        gc += 1
print(f'Successfully aligned {gc}/{len(filenameindexednonhybs)} ')

('TTPGTLALE', 'DLQTLALE', 'NOD2_E3.13729.13828.2', 'non-hybrid')
('TLNDLTGTKPVATPV', 'DSLQLTGTKPVATPV', 'NOD2_E3.10121.10155.2', 'non-hybrid')
('DLKTPAGLQVIGG', 'DLKTPAGLQVLN', 'NOD2_E3.11316.11435.2', 'non-hybrid')
('DSRGGGGNFGPGPGAGRGYNS', 'DSRGGGGNFGPGPGSNFRGGS', 'NOD2_E3.5028.5028.3', 'non-hybrid')
('SNRSYVVEYIPDAE', 'EVQSSRSGRGGNFGFG', 'NOD2_E3.4133.4133.3', 'non-hybrid')
('DLRNNFP', 'DLNRNFP', 'NOD2_E3.6072.6137.2', 'non-hybrid')
('SSELTSQLAQQESEI', 'DSRPGGYGYGYGRSR', 'NOD2_E3.3190.3222.4', 'non-hybrid')
('GDPVIHQEMIGGFR', 'DGPVIHQEMIGGFR', 'NOD2_E3.8803.8803.3', 'non-hybrid')
('DPQVEQLELGGSGP', 'DPQVEQLELGGSPG', 'NOD2_E3.12220.12676.2', 'non-hybrid')
('TASPQLELGGSPG', 'QVEQLELGGSPG', 'NOD2_E3.9045.9108.2', 'non-hybrid')
('ERGFFYTPSK', 'ERGFFYTPKS', 'NOD2_E3.5323.5391.3', 'non-hybrid')
('TTPGTLALEV', 'DLQTLALEV', 'NOD2_E3.17612.17780.2', 'non-hybrid')
('EVQITALEVAQQ', 'DLQTLALEVAQQ', 'NOD2_E3.18205.18816.2', 'non-hybrid')
('QVAQLELGGGPGQ', 'QVAQLELGGGPGAG', 'NOD2_E3.9723.9789.2',

In [10]:
gch = 0
missed_hpairs = []
for pair in hybpairs:
    if pair[0] != pair[1]:
        missed_hpairs.append(pair)
        print(pair)
    else:
        gch += 1
print(f'Successfully aligned {gch}/{len(filenameindexedhybs)}')

('TLAITASPLI', 'DLQTLALLL', 'NOD2_E3.21510.21510.2', 'hybrid')
('DPQVAQLELGGGPGAGDLQTAQLELGGGPGAG', 'DPQVAQLELGGEVEDPQVAQLELGGGPGAG', 'NOD2_E3.12771.12902.3', 'hybrid')
('LDPVNSPMTKG', 'DLPVNSPMTKG', 'NOD2_E3.7065.7065.2', 'hybrid')
('ELTNIEDL', 'DLQTLEVE', 'NOD2_E3.11427.11516.2', 'hybrid')
('SSPAAYNGETPTEKLPFPII', 'DINAYNGETPTEKLPFPII', 'NOD2_E3.18104.18104.3', 'non-hybrid')
('NAEVRKALSRQIIAEVLGSPP', 'DINAYNGETPTEKLPFPIID', 'NOD2_E3.17517.17517.3', 'non-hybrid')
('TVTVQFEANTTIGRIRFH', 'DEAPNFEANTTIGRIRFH', 'NOD2_E3.9670.9702.4', 'non-hybrid')
('DLVPRAAT', 'DLPVGRSV', 'NOD2_E3.5978.6049.2', 'non-hybrid')
('SEGAHLEHAEQEGQCRVDDK', 'DTGAGSIREAGGAFGKREKAEE', 'NOD2_E3.3808.3850.4', 'non-hybrid')
('SSNIVPQRPEGCVVAAVAARSE', 'DTGAGSIREAGGAFGKREKAEE', 'NOD2_E3.3820.3820.5', 'non-hybrid')
('LDHPGFQIRDSKGLHNVQKYKFMELN', 'DIYNFFSPLNPVRVHIEIGPDGRVTGEA', 'NOD2_E3.17254.17254.4', 'non-hybrid')
('SVAYKDEWKDLTPYGVFAGR', 'DVTKGRKFYGPEGPYGVFAGR', 'NOD2_E3.7225.7225.4', 'non-hybrid')
('SSKTVVKVEKQ', 'DSE

## 4. See what the other alignments were for the missed sets

In [11]:
# load the json
jsres = json.load(open(resultsjsonfile, 'r'))

# all the entries are <filename>_<scan_no> so go through and index them by their filename
filenameindexedhypedsearch = {}
for name, alignments in jsres.items():
    # get the id
    fname = alignments['spectrum']['id'].replace('.pkl', '')
    
    filenameindexedhypedsearch[fname] = alignments


In [12]:
# for each of the unsuccesfull non hybrids, print the others
in_top_x = 0
for nh in missed_nhpairs:
    print()
    print(f'Acutal alignment: {nh[1]} \t Hybrid: {"non" not in nh[3]} \t ID: {nh[2]}')
    print('Attempted alignemnts:')
    for alignment in filenameindexedhypedsearch[nh[2]]['alignments']:
        hybrid = 'hybrid_sequence' in alignment
        foundstring = '*' if alignment['sequence'] == nh[1] else ''
        in_top_x += 1 if alignment['sequence'] == nh[1] else 0
        print(f'{foundstring}Sequence: {alignment["sequence"]} \t Hybrid: {hybrid} \t Total score: {alignment["total_score"]} \t \
              B score: {alignment["b_score"]} \t Y score: {alignment["y_score"]}')
        

print(f'Number found in the top 5: {in_top_x}')


Acutal alignment: DLQTLALE 	 Hybrid: False 	 ID: NOD2_E3.13729.13828.2
Attempted alignemnts:
Sequence: TTPGTLALE 	 Hybrid: False 	 Total score: 9 	               B score: 6 	 Y score: 3
Sequence: TTPGTIAIE 	 Hybrid: True 	 Total score: 9 	               B score: 6 	 Y score: 3
Sequence: TTPGTLAIE 	 Hybrid: True 	 Total score: 9 	               B score: 6 	 Y score: 3
Sequence: EVQTLALE 	 Hybrid: False 	 Total score: 9 	               B score: 6 	 Y score: 3
Sequence: TLAGDLALE 	 Hybrid: True 	 Total score: 9 	               B score: 6 	 Y score: 3

Acutal alignment: DSLQLTGTKPVATPV 	 Hybrid: False 	 ID: NOD2_E3.10121.10155.2
Attempted alignemnts:
Sequence: TLNDLTGTKPVATPV 	 Hybrid: False 	 Total score: 11 	               B score: 5 	 Y score: 6
Sequence: TINDLTGTKPVATPV 	 Hybrid: False 	 Total score: 11 	               B score: 5 	 Y score: 6
Sequence: LTDNLTGTKPVATPV 	 Hybrid: False 	 Total score: 11 	               B score: 5 	 Y score: 6
Sequence: DSLQLTALKLCGHK 	 Hybrid: True 	 To

In [14]:
h_in_top_x = 0
for h in missed_hpairs:
    print()
    print(f'Acutal alignment: {h[1]} \t Hybrid: {"non" not in h[3]} \t ID: {h[2]}')
    print('Attempted alignments:')
    for alignment in filenameindexedhypedsearch[h[2]]['alignments']:
        hybrid = 'hybrid_sequence' in alignment
        foundstring = '*' if alignment['sequence'] == h[1] else ''
        h_in_top_x += 1 if alignment['sequence'] == h[1] and hybrid else 0
        print(f'{foundstring}Sequence: {alignment["sequence"]} \t Hybrid: {hybrid} \t Total score: {alignment["total_score"]} \t \
              B score: {alignment["b_score"]} \t Y score: {alignment["y_score"]}')
print(f'Number found in the top 5: {h_in_top_x}')


Acutal alignment: DLQTLALLL 	 Hybrid: True 	 ID: NOD2_E3.21510.21510.2
Attempted alignments:
Sequence: TLAITASPLI 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 3
Sequence: TLAITAPSII 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 3
Sequence: DLQTLALIL 	 Hybrid: True 	 Total score: 8 	               B score: 6 	 Y score: 2
Sequence: TLAITASPIL 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 3
Sequence: TLAITAPSIL 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 3

Acutal alignment: DPQVAQLELGGEVEDPQVAQLELGGGPGAG 	 Hybrid: True 	 ID: NOD2_E3.12771.12902.3
Attempted alignments:
Sequence: DPQVAQLELGGGPGAGDLQTAQLELGGGPGAG 	 Hybrid: True 	 Total score: 24 	               B score: 16 	 Y score: 8
*Sequence: DPQVAQLELGGEVEDPQVAQLELGGGPGAG 	 Hybrid: True 	 Total score: 19 	               B score: 10 	 Y score: 9
Sequence: DPQVAQLELGGGPVGHVHCREIICSVAF 	 Hybrid: True 	 Total score: 16 	               B scor

Sequence: QVVLGLYIVRGDIAWLVKGRGRR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: KKTEAATTAGKPEIAWLVKGRGRR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: KSRVKEEAKSPGEAWLVKGRGRR 	 Hybrid: False 	 Total score: 8 	               B score: 3 	 Y score: 5

Acutal alignment: DVSSYLEGQAAKEFIAWLVKGRGRR 	 Hybrid: False 	 ID: NOD2_E3.15511.16033.6
Attempted alignments:
Sequence: KKLISRKDASVRFIAWLVKGRGRR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: KKPMSLASGSVPAEFIAWLVKGRGRR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: KKDSELSKGLAPEFIAWLVKGRGRR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: VSCVKLLLSKGADKEAARLREERLR 	 Hybrid: False 	 Total score: 6 	               B score: 3 	 Y score: 3
Sequence: TTKLVDDGKKKPRACALLLLFPLTAQ 	 Hybrid: True 	 Total score: 6 	               B score: 3 	 Y score: 3

Acutal alignment: EGQAAKEF

Sequence: TTAPVQLYKEMVDASFIEQVAVS 	 Hybrid: True 	 Total score: 6 	               B score: 4 	 Y score: 2
Sequence: KKDKSLPVSSCESSASFIEQVAVS 	 Hybrid: True 	 Total score: 6 	               B score: 4 	 Y score: 2
Sequence: TTPGPDTEAALAKDIEAAERTIGAQ 	 Hybrid: True 	 Total score: 6 	               B score: 3 	 Y score: 3
Sequence: TTPGPDTEAALAKDIEAAATATARPS 	 Hybrid: True 	 Total score: 6 	               B score: 3 	 Y score: 3

Acutal alignment: LEYIEKLRQQAHR 	 Hybrid: False 	 ID: NOD2_E3.4503.4503.4
Attempted alignments:
Sequence: EYLLKWKAATLGHR 	 Hybrid: True 	 Total score: 8 	               B score: 3 	 Y score: 5
Sequence: KYNLGAPVAGPVLETR 	 Hybrid: True 	 Total score: 8 	               B score: 4 	 Y score: 4
Sequence: EYLLKWKFFQKR 	 Hybrid: True 	 Total score: 7 	               B score: 3 	 Y score: 4
Sequence: LEYDEKLRQQAHR 	 Hybrid: True 	 Total score: 7 	               B score: 3 	 Y score: 4
Sequence: YELDEKLRQQAHR 	 Hybrid: True 	 Total score: 7 	               B score: 3 	 Y